# Sulphur Isotope Model

## General definitions

In [1]:
import numpy as np
import pandas as pd
import ipywidgets as wdg
from plotly_default import go, graph_config
from plotly.subplots import make_subplots
from scipy.integrate import solve_bvp
# import plotly.io as pio

In [2]:
# Conversion between isotope ratios and delta notation

VCDT = 0.0441626 # vienna canyon diablo troilite standard 34S/32S ratio

def delta_to_R(delta, std=VCDT):
    '''Takes in a d34S; outputs a 34S/32S ratio'''
    return std * ((delta/1000) + 1)

def R_to_delta(R, std=VCDT):
    '''Takes in a 34S/32S ratio; outputs the d34S'''
    return 1000*((R/std) - 1)

def wt_to_M(wt, Mr=55.845, density=1.7):
    '''Converts a weight percent of a substance with molar mass Mr (default iron) into a molar concentration
    in a sediment of specified density (default 1.7 g/cm³)'''
    # g/dm³  = 1000 * g/cm³ = 1000 * density * wt% / 100 = 10 * density * wt%
    return 10. * density * wt / Mr

def M_to_wt(M, Mr=55.845, density=1.7):
    return M * Mr / (10. * density)

def ratio_to_concs(R, C):
    '''Takes a ratio of two concentrations R = a/b
    and a total concentration C = a + b
    and returns an (a, b) tuple'''
    return (C/(1 + 1/R) , C/(1+R))

# The number of seconds in 1 thousand years
s_in_ka = 1000 * 365.24 * 24 * 60 * 60

In [3]:
# Read in real data
log_df=pd.read_csv('./Data/Log_data_combined.csv', index_col=0)

cat_symbols = {'Mudstone': 'circle',
               'Carbonate': 'star',
               'Pervasively pyritised regions': 'square',
               'Bed 22': 'diamond'}
categories = list(cat_symbols.keys())

# log_df

In [4]:
# Set up widgets
w_df = pd.read_csv('./Data/GDE_parameters_v2.csv')


def create_sliders(w_df, prefix='w'):
    w_dict = {}

    for w in w_df.iterrows():

        # Make a new variable for each widget
        vars()[prefix+'_'+w[1].var_name] = wdg.FloatSlider(value = w[1].value,
                                                     min = w[1].start,
                                                     max = w[1].stop,
                                                     step = w[1].step,
                                                     description = w[1].display_name,
                                                     layout=wdg.Layout(width='100%')
                                                    )

        # Add to a dictionary of original variables and widgets
        w_dict[w[1].var_name] = vars()[prefix+'_'+w[1].var_name]
    
    return w_dict

## Solving the steady-state General Diagenetic Equation

$y$  | represents   | <div style="width:100px">value at $z=0$</div> | <div style="width:100px"> value at $z=z_\mathrm{max}$</div>
-----|--------------------------------------------------|--------------------------|-----------------------------
$y_0$|$[\mathrm{^{34}SO_4}]$                            |$[\mathrm{^{34}SO_4}]_0$  |?? $(\approx 0)$
$y_1$|$\frac{\partial[\mathrm{^{34}SO_4}]}{\partial z}$ |??                        |$0$
$y_2$|$[\mathrm{^{32}SO_4}]$                            |$[\mathrm{^{32}SO_4}]_0$  |?? $(\approx 0)$
$y_3$|$\frac{\partial[\mathrm{^{32}SO_4}]}{\partial z}$ |??                        |$0$
$y_4$|$[\mathrm{Fe^{34}S_2}]$                           |$[\mathrm{Fe^{34}S_2}]_0$ |??
$y_5$|$[\mathrm{Fe^{32}S_2}]$                           |$[\mathrm{Fe^{32}S_2}]_0$ |??

$$\frac{\partial y_0}{\partial z} = y_1$$

$$\frac{\partial y_1}{\partial z} = \left( vy_1 + \frac{k_\mathrm{MSR}\left(y_0 y_2 + y_0^2\right)}{\frac{y_2}{\alpha} + y_0} \right)/D_{34}$$

$$\frac{\partial y_2}{\partial z} = y_3$$

$$\frac{\partial y_3}{\partial z} = \left( vy_3 + \frac{k_\mathrm{MSR}\left(y_2^2 + y_0 y_2\right)}{\alpha y_0 + y_2} \right)/D_{32}$$

$$\frac{\partial y_4}{\partial z} = \frac{k_\mathrm{MSR}\left(y_0 y_2 + y_0^2\right)}{2v\left(\frac{y_2}{\alpha} + y_0\right)}$$

$$\frac{\partial y_5}{\partial z} = \frac{k_\mathrm{MSR}\left(y_2^2 + y_0 y_2\right)}{2v\left(\alpha y_0 + y_2\right)}$$

In [5]:
def solve_GDE(zs, SO4_0, py_0, d34S_SO4_0, epsilon_MSR, k_MSR, beta, D_32, v):
    '''Solves the system of general diagenetic equations listed above,
    for the array of depths in zs, and returns a 2D array of y values
    (rows for each yi; cols are values of yi for each z)'''
    
    R_SO4_0 = delta_to_R(d34S_SO4_0) 
    a_MSR = delta_to_R(d34S_SO4_0 - epsilon_MSR)/R_SO4_0 # MSR fractionation factor    
    R_py_0 = delta_to_R(d34S_SO4_0)*a_MSR # add ability to tweak in future?
    
    # weight-dependent diffusion coefficients (Wortmann & Chernyavsky, 2011)
    D_34 = D_32 * (98/96)**(-beta)
    
    SO4_34_0 , SO4_32_0 = ratio_to_concs(R_SO4_0, SO4_0)
    py_34_0  , py_32_0  = ratio_to_concs(R_py_0, py_0)
    
    # Define the ODEs and boundary conditions for the chosen parameters
    def SS_ODEs(x,y):
        '''The right-hand side of the steady state general diagenetic equations'''
        return np.vstack([
            y[1,:], # = dy[0,:]/dz
            (v*y[1,:] + k_MSR*(y[0,:]*y[2,:] + y[0,:]**2)/((y[2,:]/a_MSR) + y[0,:]))/D_34, # = dy[1,:]/dz
            y[3,:], # = dy[2,:]/dz
            (v*y[3,:] + k_MSR*(y[2,:]**2 + y[0,:]*y[2,:])/((a_MSR*y[0,:]) + y[2,:]))/D_32, # = dy[3,:]/dz
            k_MSR*(y[0,:]*y[2,:] + y[0,:]**2)/(2*v*((y[2,:]/a_MSR) + y[0,:])), # = dy[4,:]/dz
            k_MSR*(y[2,:]**2 + y[0,:]*y[2,:])/(2*v*((a_MSR*y[0,:]) + y[2,:])) # = dy[5,:]/dz
        ])    
    
    def SS_bcs(ya, yb):
        '''The GDE boundary condition residuals, i.e. all the terms involving ya should be
        zero at z=0, and all the terms involving yb should be zero at z=max_z'''
        return np.array([
            ya[0] - SO4_34_0,
            yb[1] - 0,
            ya[2] - SO4_32_0,
            yb[3] - 0,
            ya[4] - py_34_0,
            ya[5] - py_32_0
        ])
    
    # Set up initial array of concentrations to iterate the solution from
    y0s = np.ones((6,len(zs)))
    
    # Use SciPy's boundary value problem solver to solve the steady state ODEs
    SS_sol = solve_bvp(SS_ODEs, SS_bcs, zs, y0s)
    
    # Extract the yi from the solution and return it
    return SS_sol.y



def update_GDE_plot(plot, SO4_0, py_0, d34S_SO4_0, epsilon_MSR, k_MSR, beta, D_32, v, max_z, combine_isotopes, dz=0.01, plot_time=False):
    
    zs = np.arange(0, max_z, dz) # values of z to plot over.
    
    concs = solve_GDE(zs, SO4_0, py_0, d34S_SO4_0, epsilon_MSR, k_MSR, beta, D_32, v)
    
    SO4_34s = concs[0,:]
    SO4_32s = concs[2,:]
    py_34s  = concs[4,:]
    py_32s  = concs[5,:]
    
    
    # Convert to concentrations and d34S
    SO4s = SO4_34s + SO4_32s
    pys = py_34s + py_32s

    d34S_SO4s = []
    d34S_pys = []
    
    isotope_tolerance = 1e-12 # mol/dm^3: if numbers below this point, add none to d34S list
    max_SO4_d34S = 100 # don't plot any sulphate isotopes larger than this
    # (n.b. this doesn't affect calculations of other quantities)
    SO4_d34S_valid = True
    
    for SO4_34, SO4_32, py_34, py_32 in zip(SO4_34s, SO4_32s, py_34s, py_32s):
        if (SO4_32 > isotope_tolerance) and SO4_d34S_valid:
            d34S_SO4 = R_to_delta(SO4_34/SO4_32)
            d34S_SO4s.append(d34S_SO4)
            if d34S_SO4 > max_SO4_d34S: # once threshold is exceeded stop plotting d34S_SO4
                SO4_d34S_valid = False
        else:
            d34S_SO4s.append(None)
            
        if py_32 > isotope_tolerance:
            d34S_pys.append(R_to_delta(py_34/py_32))
        else:
            d34S_pys.append(None)
            
    if plot_time:
        zs = zs/v
        plot.update_layout(yaxis_title='Time after burial, <i>t</i> / ka')
    else:
        plot.update_layout(yaxis_title='Depth, <i>z</i> / m')
    
    plot.update_traces(x=SO4s, y=zs,visible=combine_isotopes,  selector=dict(name='[SO<sub>4</sub>]'))
    plot.update_traces(x=pys, y=zs, visible=combine_isotopes, selector=dict(name='[FeS<sub>2</sub>]'))
    plot.update_traces(x=d34S_SO4s, y=zs, visible=combine_isotopes, selector=dict(name='δ<sup>34</sup>S<sub>SO4</sub>'))
    plot.update_traces(x=d34S_pys, y=zs, visible=combine_isotopes, selector=dict(name='δ<sup>34</sup>S<sub>py</sub>'))
    
    plot.update_traces(x=SO4_34s, y=zs, visible= not combine_isotopes, selector=dict(name='[<sup>34</sup>SO<sub>4</sub>]'))
    plot.update_traces(x=SO4_32s, y=zs, visible= not combine_isotopes, selector=dict(name='[<sup>32</sup>SO<sub>4</sub>]'))
    plot.update_traces(x=py_34s, y=zs, visible= not combine_isotopes, selector=dict(name='[Fe<sup>34</sup>S<sub>2</sub>]'))
    plot.update_traces(x=py_32s, y=zs, visible= not combine_isotopes, selector=dict(name='[Fe<sup>32</sup>S<sub>2</sub>]'))
    
    plot.show()
    
    # final_py, final_d34S = get_py_d34S(SO4_0, py_0, d34S_SO4_0, epsilon_MSR, k_MSR, beta, D_32, v, dz, max_z)
    print('Final [py]: {:.4f}'.format(pys[-1]))
    print('Final d34S: {:.4f}'.format(d34S_pys[-1]))

#### Parameters

Quantity | Name in Code | Value | Unit | Reference
----:----|------:-------|------:|:-----|----:-----
Sulphate concentration in seawater | `SO4_0` | 0.0105 | mol dm<sup>−3</sup> | Horita *et al.* (2002)
Concentration of authigenic pyrite in sediment | `py_0` | ?? | mol dm<sup>−3</sup> |
Isotopic composition of seawater sulphate | `d34S_SO4_0` | 15 ~ 17 | ‰ | Kampschulte & Strauss (2004)
Microbial sulphate reduction fractionation amount at surface |`epsilon_MSR` | 0 ~ 66 | ‰ | Fike *et al.* (2016)
Rate constant for sulphate reduction | `k_MSR` | 7 | ka<sup>−1</sup> | Egger *et al.* (2016)
Amount of mass dependency for diffusion coefficients | `beta` | 0 ~ 0.05 | (dimensionless) | Wortmann & Chernyavsky (2011)
Diffusivity of <sup>32</sup>SO<sub>4</sub> | `D_32` | ≈ 0.126 | m<sup>2 </sup>ka<sup>−1</sup> | Jørgensen (1979)
Sedimentation rate | `v` | ≈ 0.08 | m ka<sup>−1</sup> | Gallois (2004)
Max depth to run simulation to | `max_z` | (until δ<sup>34</sup>S<sub>py</sub> stabilises)| m |

In [6]:
GDE_plot = make_subplots(rows=1, cols=2, shared_yaxes=True, horizontal_spacing=0.01)
GDE_plot.add_trace(go.Scatter(name='[SO<sub>4</sub>]', marker_color='RoyalBlue', mode='lines'), row=1, col=1)
GDE_plot.add_trace(go.Scatter(name='[FeS<sub>2</sub>]', marker_color='MediumSeaGreen', mode='lines'), row=1, col=1)
GDE_plot.add_trace(go.Scatter(name='δ<sup>34</sup>S<sub>SO4</sub>', marker_color='Navy', mode='lines'), row=1, col=2)
GDE_plot.add_trace(go.Scatter(name='δ<sup>34</sup>S<sub>py</sub>', marker_color='DarkGreen', mode='lines'), row=1, col=2)

GDE_plot.add_trace(go.Scatter(name='[<sup>34</sup>SO<sub>4</sub>]', mode='lines'), row=1, col=1)
GDE_plot.add_trace(go.Scatter(name='[Fe<sup>34</sup>S<sub>2</sub>]', mode='lines'), row=1, col=1)
GDE_plot.add_trace(go.Scatter(name='[<sup>32</sup>SO<sub>4</sub>]', mode='lines'), row=1, col=1)
GDE_plot.add_trace(go.Scatter(name='[Fe<sup>32</sup>S<sub>2</sub>]', mode='lines'), row=1, col=1)

GDE_plot.update_layout(
    yaxis=dict(autorange='reversed', rangemode='tozero', title='Depth, <i>z</i> / m'),
    hovermode='y',
    margin=dict(t=60),
    width=1000, height=600
    )

GDE_plot.update_xaxes(title_text='Concentration / mol dm<sup>−3</sup>', row=1, col=1)
GDE_plot.update_xaxes(title_text='δ<sup>34</sup>S (‰)', row=1, col=2)

# create widgets
w_d = create_sliders(w_df, prefix='w')

w_combine = wdg.Checkbox(value=True, description='Combine isotopes')
w_plot_time = wdg.Checkbox(value=False, description='Plot time on y axis')

# set up widget layout
GDE_ui = wdg.VBox([wdg.HBox([w_d['SO4_0'], w_d['d34S_SO4_0']]),
                   wdg.HBox([w_d['py_0'], w_d['v']]),
                   wdg.HBox([w_d['epsilon_MSR'], w_d['k_MSR']]),
                   wdg.HBox([w_d['D_32'], w_d['beta']]),
                   wdg.HBox([w_d['max_z'], w_d['dz']]),
                   wdg.HBox([w_combine, w_plot_time])
                  ])

GDE_args = {**w_d, 
            'plot': wdg.fixed(GDE_plot),
            'combine_isotopes':w_combine,
            'plot_time': w_plot_time}

GDE_output = wdg.interactive_output(update_GDE_plot, GDE_args)

display(GDE_ui, GDE_output)

Output()

In [7]:
# GDE_plot.update_xaxes(row=1, col=1, range=[0,0.3])

In [8]:
#GDE_plot.write_image('./images/GDE_example.svg')

In [9]:
def get_py_d34S(SO4_0, py_0, d34S_SO4_0, epsilon_MSR, k_MSR, beta, D_32, v, dz, max_z):
    '''Returns the final pyrite concentration and its d34S given the appropriate initial conditions'''
    
    zs = np.arange(0, max_z, dz) # values of z to plot over.
    
    concs = solve_GDE(zs, SO4_0, py_0, d34S_SO4_0, epsilon_MSR, k_MSR, beta, D_32, v)
    
    # Final amounts of py32 and py34
    py_34  = concs[4,-1]
    py_32  = concs[5,-1]
    
    py = py_34 + py_32
    d34S = R_to_delta(py_34/py_32)
    
    return py, d34S


def update_var_plot(plot, varying, vstart, vstop, vstep, SO4_0, py_0, d34S_SO4_0, epsilon_MSR, k_MSR, beta, D_32, v, dz, max_z, plot_auth, plot_noauth):
    
    # Create dictionary of parameters
    params = {}
    for w in w_df.iterrows():
        param = w[1].var_name
        params[param] = locals()[param] # locals() get the actual variables put into the function
    
#     print(params)
    
    pys = []
    d34Ss = []
    texts = []
    
    vary_range = np.arange(vstart, vstop, vstep)
    
    for var in vary_range:
        
        # Update the appropriate parameter to its value in the iterations
        params[varying] = var
        
        py, d34S = get_py_d34S(**params)
        
        pys.append(py)
        d34Ss.append(d34S)
        texts.append('{}: {:.5f}'.format(varying, var))
        
#         print('{}: {:.5f}'.format(varying, vars()[varying]))
#         print('[py]: {:.4f}'.format(py))
#         print('d34S: {:.4f}\n'.format(d34S))
        
    plot.update_traces(x=pys, y=d34Ss, text=texts, marker_color=vary_range, selector=dict(name='Model'))
    
    if plot_auth:
        plot.update_traces(x=[py_0], y=[d34S_SO4_0 - epsilon_MSR],
                           text=['{:.2f} wt% Fe<sub>py, 0</sub>'.format(M_to_wt(py_0))],
                           visible=True, selector=dict(name='Authigenic Pyrite'))
    else:
        plot.update_traces(visible=False, selector=dict(name='Authigenic Pyrite'))

        
    if plot_noauth:
        # recalculate model with no authigenic pyrite
        pys_na = []
        d34Ss_na = []
        texts_na = []
        
        params_na = params
        params_na['py_0'] = 0 # set to no authigenic pyrite

        for var in vary_range:

            # Update the appropriate parameter to its value in the iterations
            params_na[varying] = var

            py, d34S = get_py_d34S(**params_na)

            pys_na.append(py)
            d34Ss_na.append(d34S)
            texts_na.append('{}: {:.5f}'.format(varying, var))
        
        plot.update_traces(x=pys_na, y=d34Ss_na, text=texts_na, marker_color=vary_range,
                           visible=True, selector=dict(name='Model (no py<sub>0</sub>)'))
    else:
        plot.update_traces(visible=False, selector=dict(name='Model (no py<sub>0</sub>)'))
        
    # change colour bar title
    plot.update_layout(coloraxis_colorbar_title=list(w_df[w_df.var_name==varying].display_name)[0])
    
    plot.show()
    
    print()

In [10]:
# Set up plot
var_plot=go.Figure()

var_plot.update_layout(
    yaxis=dict(title='δ<sup>34</sup>S<sub>py</sub> (‰)'),
    xaxis=dict(title='[FeS<sub>2</sub>] / mol dm<sup>−3</sup>', rangemode='tozero'),
    legend=dict(yanchor="top", y=1, xanchor="left", x=1.2),
    margin=dict(l=100),
    width=1000, height=600,
    coloraxis=dict(colorscale='viridis_r', colorbar_ticks='outside')
    )

# Add the real data
for category in categories:
    cat_df = log_df[log_df.category == category]
    var_plot.add_trace(go.Scatter(x = cat_df.Fe_py.apply(wt_to_M), y=cat_df.d34S,
                                  text = cat_df.name,
                                  mode='markers',
                                  marker_color='Blue',
                                  marker_opacity=0.3,
                                  marker_symbol=cat_symbols[category],
                                  name=category))

var_plot.add_trace(go.Scatter(name='Model', mode='lines+markers', marker_coloraxis='coloraxis',
                              marker_line_color='DarkGreen', marker_line_width=1, line_color='DarkGreen'))
var_plot.add_trace(go.Scatter(name='Model (no py<sub>0</sub>)', mode='lines+markers', marker_coloraxis='coloraxis',
                              marker_line_color='Orange', marker_line_width=1, line_color='Orange'))
var_plot.add_trace(go.Scatter(name='Authigenic Pyrite', mode='markers',
                              marker_color='Purple', marker_symbol='square'))

# create widgets
field_layout = wdg.Layout(width='200px')

v_d = create_sliders(w_df, prefix='v')
v_varying=wdg.Dropdown(options=w_df.var_name, description='Vary', layout=field_layout, value='v')
v_vstart=wdg.FloatText(value=0.005, step=0.01, description='from', layout=field_layout)
v_vstop=wdg.FloatText(value=0.2, step=0.1, description='to', layout=field_layout)
v_vstep=wdg.FloatText(value=0.005, step=0.01, description='step', layout=field_layout)
v_plot_auth = wdg.Checkbox(value=True, description='Plot authigenic pyrite')
v_plot_noauth = wdg.Checkbox(value=False, description='Plot curve w/o authigenic pyrite')

# set up widget layout
var_ui = wdg.VBox([wdg.HBox([v_d['SO4_0'], v_d['d34S_SO4_0']]),
                   wdg.HBox([v_d['py_0'], v_d['v']]),
                   wdg.HBox([v_d['epsilon_MSR'], v_d['k_MSR']]),
                   wdg.HBox([v_d['D_32'], v_d['beta']]),
                   wdg.HBox([v_d['max_z'], v_d['dz']]),
                   wdg.HBox([v_varying, v_vstart, v_vstop, v_vstep]),
                   wdg.HBox([v_plot_auth, v_plot_noauth])
                  ])

var_args = {**v_d, 
            'plot': wdg.fixed(var_plot),
            'varying':v_varying,
            'vstart':v_vstart,
            'vstop':v_vstop,
            'vstep':v_vstep,
            'plot_auth': v_plot_auth,
            'plot_noauth': v_plot_noauth
           }

var_output = wdg.interactive_output(update_var_plot, var_args)

display(var_ui, var_output)

Output()

In [11]:
# var_plot.write_image('./images/varying_SO4.svg')

### Modelling Iron Speciation with the same parameters

In [12]:
def update_Fe_spec(plot, SO4_0, py_0, d34S_SO4_0, epsilon_MSR, k_MSR, beta, D_32, v, dz, max_z, Fe_tot, plot_residuals):
    
    # Calculate how much pyrite is produced
    py, d34S = get_py_d34S(SO4_0, py_0, d34S_SO4_0, epsilon_MSR, k_MSR, beta, D_32, v, dz, max_z)
    Fe_py = M_to_wt(py)
    Fe_py_0 = M_to_wt(py_0)
    
    # update model line
    HR_tot_0 = np.linspace(0.01,1,50) # Increase last number for higher resolution
    
    HR_0 = HR_tot_0 * Fe_tot
    py_HR_t = py/HR_0
    
    plot.update_traces(selector=dict(name='model (const. total Fe)'),
                       x=HR_tot_0, y=py_HR_t)
    
    
    # update data points
    for category in categories:
        cat_df = log_df[log_df.category == category]
        
        py_HR_points = list(Fe_py/cat_df['Fe_HR'])
        
        py_0_HR = list(Fe_py_0/cat_df['Fe_HR'])
        
        for i in range(len(py_HR_points)):
            if py_HR_points[i] > 1:
                py_HR_points[i] = 1

        plot.update_traces(selector=dict(name=category+' (modelled)'),
                           x=cat_df['HR_to_tot'], y=py_HR_points)
        
        plot.update_traces(selector=dict(name=category+' (pre-diagenesis)'),
                           x=cat_df['HR_to_tot'], y=py_0_HR)
        
        if plot_residuals:
            plot.update_traces(selector=dict(name=category+' (residuals)'), visible=True,
                               x=cat_df['HR_to_tot'], y=cat_df['py_to_HR'] - py_HR_points)
        else:
            plot.update_traces(selector=dict(name=category+' (residuals)'), visible=False)
    
    if plot_residuals:
        plot.update_layout(yaxis_range=[-0.5,1])
    else:
        plot.update_layout(yaxis_range=[0,1])
    
    plot.show(config=graph_config)

In [13]:
# Modelling pyritisation of real data
psn_model=go.FigureWidget()

# for HR_to_tot in list(log_df.HR_to_tot):
#     psn_model.add_vline(HR_to_tot, line_width=0.5, line_color='Lavender')

for category in categories: 
    
    cat_df = log_df[log_df.category == category]

    graph_texts = []
    for d in cat_df.iterrows():
        graph_texts.append(d[1]['name'])
    
    # Current locations of samples in Fe speciation space
    
    psn_model.add_trace(go.Scatter(mode='markers',
                                   marker_symbol=cat_symbols[category],
                                   marker_color='Purple',
                                   marker_opacity=0.3,
                                   name=category+' (pre-diagenesis)',
                                   text=graph_texts,
                                   legendgroup=category))
    
    psn_model.add_trace(go.Scatter(mode='markers',
                                   marker_symbol=cat_symbols[category],
                                   marker_color='Green',
                                   name=category+' (modelled)',
                                   text=graph_texts,
                                   legendgroup=category))

    psn_model.add_trace(go.Scatter(x=cat_df['HR_to_tot'], y=cat_df['py_to_HR'],
                                   mode='markers',
                                   marker_symbol=cat_symbols[category],
                                   marker_color='Blue',
                                   marker_opacity=0.3,
                                   name=category+' (post-diagenesis)',
                                   text=graph_texts,
                                   legendgroup=category))    
    
    psn_model.add_trace(go.Scatter(mode='markers',
                                   marker_symbol=cat_symbols[category],
                                   marker_color='FireBrick',
                                   name=category+' (residuals)',
                                   text=graph_texts,
                                   legendgroup=category))
    
    
psn_model.add_trace(go.Scatter(name='model (const. total Fe)',
                               line_color='LightGreen'))

psn_model.update_layout(xaxis_range=[0,1], yaxis_range=[0,1],
                        width=900, height=600)
psn_model.add_hline(0.8, line_width=1, line_dash='dash')
psn_model.add_vline(0.38, line_width=1, line_dash='dash')
psn_model.update_layout(xaxis_title='[Fe<sub>HR</sub>]/[Fe<sub>tot</sub>]',
                        yaxis_title='[Fe<sub>py</sub>]/[Fe<sub>HR</sub>]',
                        legend=dict(#itemsizing='constant',
                                    font_size=11))


# Create widgets
s_d = create_sliders(w_df, prefix='s')
s_Fe_tot=wdg.FloatSlider(value=4, min=0, max=15, description='[Fe<sub>tot</sub>]', layout=wdg.Layout(width='100%')) # Total Fe for simple model curve
s_plot_residuals=wdg.Checkbox(value=False, description='Plot Residuals')

spec_ui = wdg.VBox([wdg.HBox([s_d['SO4_0'], s_d['d34S_SO4_0']]),
                   wdg.HBox([s_d['py_0'], s_d['v']]),
                   wdg.HBox([s_d['epsilon_MSR'], s_d['k_MSR']]),
                   wdg.HBox([s_d['D_32'], s_d['beta']]),
                   wdg.HBox([s_d['max_z'], s_d['dz']]),
                   wdg.HBox([s_Fe_tot, s_plot_residuals])])

spec_args={**s_d, 'Fe_tot': s_Fe_tot, 'plot_residuals': s_plot_residuals, 'plot': wdg.fixed(psn_model)}

spec_out = wdg.interactive_output(update_Fe_spec, spec_args)

display(spec_ui, spec_out)

Output()

## Rayleigh Fractionation Model (no diffusion)

In [14]:
def r_MSR(t, SO4, k_MSR):
    '''Define the rate of microbial sulphate reduction, in moles of S per dm³ per ka'''
    return k_MSR*SO4

# ignoring sulphate diffusion (and thus sediment burial) for now.


def ratio_update(R_old, conc_old, R_added, conc_added):
    '''calculates the new isotope ratio after adding a substance with the isotope ratio R_added at the concentration conc_added to a substance at conc_old with the isotope ratio R_old'''
    S32_old = conc_old/(1+R_old)
    S32_added = conc_added/(1+R_added)
    return ((conc_old - S32_old) + (conc_added - S32_added))/(S32_old + S32_added)



def propagate_model(SO4_0, py_0, R_SO4_0, a_MSR, k_MSR, dt, max_t):
    '''Finds the evolution of sulphate and pyrite concentrations and their isotope ratios over time, given:
    - initial concentrations SO4_0 and py_0 (mol dm⁻³)
    - initial sulphate isotope ratio R_SO4
    - sulphate reduction fractionation factor, a_MSR
    - rate of sulphate reduction drop-off, k_MSR,
    - simulation timestep, dt (ka)
    - max simulation time (ka)
    
    returns times, SO4s, pys, d34S_SO4s, d34S_pys'''
    
    # initialise values
    SO4 = SO4_0
    py = py_0
    R_SO4 = R_SO4_0
    R_py = a_MSR*R_SO4_0

    # initialise lists
    SO4s = []
    pys = []
    R_SO4s = []
    R_pys = []

    times = np.arange(0, max_t, dt)

    for t in times:
        # print(t)

        # Add current values to lists
        SO4s.append(SO4)
        pys.append(py)
        R_SO4s.append(R_SO4)
        R_pys.append(R_py)

        # Calculate concentration changes  
        d_SO4 = -r_MSR(t, SO4, k_MSR) * dt
        d_py = -0.5*d_SO4

        # Calculate new isotope ratios:
        # R_py = (R_py*py + a_MSR*d_py)/(py + d_py)
        R_py = ratio_update(R_py, py, a_MSR*R_SO4, d_py)
        R_SO4 = ratio_update(R_SO4, SO4, a_MSR*R_SO4, d_SO4)

        # Update concentrations
        SO4 += d_SO4
        py += d_py
        
        if SO4 <= 0:#.0001:
            break # stops ridiculous things happening with tiny/negative concentrations

        
    d34S_SO4s = [R_to_delta(R) for R in R_SO4s]
    d34S_pys = [R_to_delta(R) for R in R_pys]
    
    return times, SO4s, pys, d34S_SO4s, d34S_pys #, R_SO4s, R_pys

In [15]:
# define model parameters
SO4_0 = 1. # mol dm⁻³
py_0 = 0.
d34S_SO4_0 = 20. # seawater sulphate isotopes

dt = 0.01 # ka 
max_t = 5. # ka

def update_S_plot(plot, SO4_0, py_0, d34S_SO4_0, epsilon_MSR, k_MSR, max_t, dt=0.01):

    # epsilon_MSR = 70 # ‰ (at surface)
    R_SO4_0 = delta_to_R(d34S_SO4_0) 
    a_MSR = delta_to_R(d34S_SO4_0 - epsilon_MSR)/R_SO4_0 # MSR fractionation factor
    
    times, SO4s, pys, d34S_SO4s, d34S_pys = propagate_model(SO4_0, py_0, R_SO4_0, a_MSR, k_MSR, dt, max_t)
    
#     rayleigh = [R_to_delta(R_SO4_0 * (SO4/SO4_0)**(a_MSR - 1)) for SO4 in SO4s]
    
#     plot.update_traces(x=rayleigh, y=times, selector=dict(name='Rayleigh Fractionation'))
    plot.update_traces(x=SO4s, y=times, selector=dict(name='[SO<sub>4</sub>]'))
    plot.update_traces(x=pys, y=times, selector=dict(name='[FeS<sub>2</sub>]'))
    plot.update_traces(x=d34S_SO4s, y=times, selector=dict(name='δ<sup>34</sup>S<sub>SO4</sub>'))
    plot.update_traces(x=d34S_pys, y=times, selector=dict(name='δ<sup>34</sup>S<sub>py</sub>'))
    
    plot.show()

In [16]:
model_plot = go.Figure()
model_plot.add_trace(go.Scatter(xaxis='x1', name='[SO<sub>4</sub>]', mode='lines'))
model_plot.add_trace(go.Scatter(xaxis='x1', name='[FeS<sub>2</sub>]', mode='lines'))
model_plot.add_trace(go.Scatter(xaxis='x2', name='δ<sup>34</sup>S<sub>SO4</sub>', mode='lines'))
model_plot.add_trace(go.Scatter(xaxis='x2', name='δ<sup>34</sup>S<sub>py</sub>', mode='lines'))
#model_plot.add_trace(go.Scatter(xaxis='x2', name='Rayleigh Fractionation'))

model_plot.update_layout(
    yaxis=dict(autorange='reversed', rangemode='tozero', title='Time, <i>t</i> / ka'),
    xaxis1=dict(title='Concentration / mol dm<sup>−3</sup>', side='top'),
    xaxis2=dict(title='δ<sup>34</sup>S (‰)', side='bottom', overlaying='x1'),
    margin=dict(t=60),
    width=800, height=600
    )

sld_width = wdg.Layout(width='75%')

wdg.interact(update_S_plot,
             plot=wdg.fixed(model_plot),
             SO4_0=wdg.FloatSlider(value=1, min=0, max=2, step=0.01, layout=sld_width),
             py_0=wdg.FloatSlider(value=0, min=0, max=1, step=0.01, layout=sld_width),
             d34S_SO4_0=wdg.FloatSlider(value=20, min=-50, max=50, step=1, layout=sld_width),
             epsilon_MSR=wdg.FloatSlider(value=70, min=0, max=100, step=1, layout=sld_width),
             k_MSR=wdg.FloatSlider(value=1, min=0.01, max=2, step=0.01, layout=sld_width),
             max_t=wdg.FloatSlider(value=10, min=0, max=50, step=0.1, layout=sld_width),
             dt=wdg.fixed(0.01));

interactive(children=(FloatSlider(value=1.0, description='SO4_0', layout=Layout(width='75%'), max=2.0, step=0.…

In [17]:
log_df=pd.read_csv('./Data/Log_data_combined.csv')

In [18]:
def update_py_plot(plot, SO4_0s, py_0, d34S_SO4_0, epsilon_MSR, k_MSR, max_t, dt=0.01):
    
    R_SO4_0 = delta_to_R(d34S_SO4_0) 
    a_MSR = delta_to_R(d34S_SO4_0 - epsilon_MSR)/R_SO4_0 # MSR fractionation factor
    
    final_pys = []
    final_d34Ss = []
    
    for SO4_0 in SO4_0s:
    
        times, SO4s, pys, d34S_SO4s, d34S_pys = propagate_model(SO4_0, py_0, R_SO4_0, a_MSR, k_MSR, dt, max_t)
        
        final_pys.append(pys[-1])
        final_d34Ss.append(d34S_pys[-1])
    
    plot.update_traces(selector=dict(name='Model'), x=final_pys, y=final_d34Ss, text=SO4_0s)
    
    plot.show()

In [19]:
py_plot = go.Figure()

py_plot.add_trace(go.Scatter(x=log_df.Fe_py.apply(wt_to_M), y=log_df.d34S, name='Data', mode='markers'))
py_plot.add_trace(go.Scatter(name='Model', mode='lines'))

py_plot.update_layout(xaxis_title='[FeS<sub>2</sub>] / mol dm<sup>−3</sup>',
                      xaxis_rangemode='nonnegative',
                      yaxis_title='δ<sup>34</sup>S<sub>py</sub>',
                      # yaxis_range=[log_df.d34S.min(), log_df.d34S.max()],
                      width=800, height=700)

SO4_0s = np.arange(0.1, 3.0, 0.05)


wdg.interact(update_py_plot,
             plot=wdg.fixed(py_plot),
             SO4_0s=wdg.fixed(SO4_0s),
             py_0=wdg.FloatSlider(value=0., min=0, max=1, step=0.01, layout=sld_width),
             d34S_SO4_0=wdg.FloatSlider(value=-2, min=-50, max=50, step=1, layout=sld_width),
             epsilon_MSR=wdg.FloatSlider(value=90, min=0, max=100, step=1, layout=sld_width),
             k_MSR=wdg.FloatSlider(value=1, min=0.01, max=2, step=0.01, layout=sld_width),
             max_t=wdg.FloatSlider(value=10, min=0, max=50, step=0.1, layout=sld_width),
             dt=wdg.fixed(0.01));

interactive(children=(FloatSlider(value=0.0, description='py_0', layout=Layout(width='75%'), max=1.0, step=0.0…

## Solving the General Diagenetic Equation (old method; no isotopes)

In [20]:
from scipy.integrate import odeint

In [21]:
# for bulk sulphate

def SS_ODEs_old(x, z, D, v, k_MSR):
    '''The system of ODEs that need to be solved to get the steady state concentrations'''
    return[
           x[1], # = x[0]' = d[SO₄]/dz (Eq. a)
           (v*x[1] + k_MSR*x[0])/D, # = x[1]' = d²[SO₄]/dz² (Eq. b)
           k_MSR * x[0] / (2*v) # = x[2]' = d[py]/dz
          ]


def SO4_ana(z, D, v, r, SO4_0):
    decay_const = (np.sqrt(v**2+4*D*r)-v)/(2*D)
    return SO4_0*np.exp(-decay_const*z)
    

def update_GDE_plot_old(plot, SO4_0, py_0, d34S_SO4_0, epsilon_MSR, k_MSR, D, v, max_z, dz=0.01):
    
    zs = np.arange(0, max_z, dz) # values of z to plot over.
    
    initial_conditions = [SO4_0, # [SO₄] at surface
                          -SO4_0*(np.sqrt(v**2+4*D*k_MSR)-v)/(2*D), # d[SO₄]/dz at surface
                          py_0 # [py] at surface
                         ]
    
    SS_sol = odeint(SS_ODEs_old, y0=initial_conditions, t=zs, args=(D, v, k_MSR))
    
    SO4s = SS_sol[:,0]
    pys = SS_sol[:,2]
    
    plot.update_traces(x=SO4s, y=zs, selector=dict(name='[SO<sub>4</sub>]'))
    plot.update_traces(x=pys, y=zs, selector=dict(name='[FeS<sub>2</sub>]'))
    
    SO4s_ana = SO4_ana(zs,D,v,k_MSR,SO4_0)
    plot.update_traces(x=SO4s_ana, y=zs, selector=dict(name='analytical'))
    
    
    plot.show()

### Numerical transient solution (broken)

In [22]:
def deriv(array, i, dx):
    '''Numerically computes the derivative of a function
    (input as an array of f(x) points, with spacing dx),
    using the symmetric difference quotient, at the ith point.'''
    if i == 0:
        return (array[1] - array[0])/dx
    elif i < 0:
        raise IndexError
    else:
        try:
            return (array[i+1] - array[i-1])/(2*dx)
        except IndexError: # if we run out of space in the array, just use the asymmetric difference quotient
            return (array[i] - array[i-1])/dx

    
def deriv2(array, i, dx):
    '''Numerically computes the second derivative of a function
    (input as an array of f(x) points, with spacing dx),
    using the symmetric difference quotient, at the ith point.'''
    if i == 0:
        return (array[2] -2*array[1] + array[0])/(dx**2)
    elif i < 0:
        raise ValueError
    else:
        try:
            return (array[i+1] - 2*array[i] + array[i-1])/(dx**2)
        except IndexError:# if we run out of space in the array, assume the second derivative is the same as 1 step before
            return (array[i] -2*array[i-1] + array[i-2])/(dx**2)
    

def update_time_plot(plot, SO4_0, py_0, d34S_SO4_0, epsilon_MSR, k_MSR, D, v, max_z, time, dz=0.01, dt=0.1):
    
    zs = np.arange(0, max_z, dz) # values of z to plot over.
    times = np.arange(0, time, dt) # timesteps
    
    # Set initial conditions
    SO4s = np.zeros(len(zs))
    # SO4s = np.exp(-1*zs)
    
    # Update concentration 
    for t in times:
        
        SO4s[0] = SO4_0
        d_SO4s = [0]
        
        for i in range(1, int(max_z/dz)):
            
            d_SO4s.append(D*deriv2(SO4s, i, dz) - v*deriv(SO4s, i, dz) - k_MSR*SO4s[i])
        
        SO4s += np.array(d_SO4s)*dt
        
    
    # Plot final traces
    plot.update_traces(x=SO4s, y=zs, selector=dict(name='[SO<sub>4</sub>]'))
    
    plot.show()

In [23]:
time_plot = go.Figure()
time_plot.add_trace(go.Scatter(xaxis='x1', name='[SO<sub>4</sub>]', mode='lines'))
time_plot.add_trace(go.Scatter(xaxis='x1', name='[FeS<sub>2</sub>]', mode='lines'))
time_plot.add_trace(go.Scatter(xaxis='x2', name='δ<sup>34</sup>S<sub>SO4</sub>', mode='lines'))
time_plot.add_trace(go.Scatter(xaxis='x2', name='δ<sup>34</sup>S<sub>py</sub>', mode='lines'))

time_plot.update_layout(
    yaxis=dict(autorange='reversed', rangemode='tozero', title='Depth, <i>z</i> / m'),
    xaxis1=dict(title='Concentration / mol dm<sup>−3</sup>', side='top', range=[0,2]),
    xaxis2=dict(title='δ<sup>34</sup>S (‰)', side='bottom', overlaying='x1'),
    margin=dict(t=60),
    width=800, height=600
    )

sld_width = wdg.Layout(width='75%')

wdg.interact(update_time_plot,
             plot=wdg.fixed(time_plot),
             SO4_0=wdg.FloatSlider(value=1, min=0, max=2, step=0.01, layout=sld_width),
             py_0=wdg.FloatSlider(value=0, min=0, max=1, step=0.01, layout=sld_width),
             d34S_SO4_0=wdg.FloatSlider(value=20, min=-50, max=50, step=1, layout=sld_width),
             epsilon_MSR=wdg.FloatSlider(value=70, min=0, max=100, step=1, layout=sld_width),
             k_MSR=wdg.FloatSlider(value=1, min=0, max=2, step=0.01, layout=sld_width),
             v=wdg.FloatSlider(value=1, min=0., max=2, step=0.01, layout=sld_width),
             D=wdg.FloatSlider(value=1, min=0., max=2, step=0.01, layout=sld_width),
             max_z=wdg.FloatSlider(value=10, min=0, max=50, step=0.1, layout=sld_width),
             time=wdg.FloatSlider(value=0, min=0, max=50, step=0.1, layout=sld_width),
             dz=wdg.fixed(0.01),
             dt=wdg.fixed(0.1));

interactive(children=(FloatSlider(value=1.0, description='SO4_0', layout=Layout(width='75%'), max=2.0, step=0.…

### Testing of solve_bvp

In [24]:
n = 100 # number of points
x = np.linspace(0,3.2,n)
y0 = np.zeros((3,n))

test_plot = go.Figure()
test_plot.add_trace(go.Scatter(name='y0'))
test_plot.add_trace(go.Scatter(name='y1'))
test_plot.add_trace(go.Scatter(name='y2'))
test_plot.update_layout(width=700, height=500)

def test_bvp(plot, a, b, r):
    
    def f(x, y):
        "right hand side of ODE"
        return np.vstack([y[1,:],
                          r*y[0,:],
                          y[1,:]*y[0,:]])

    def bc(ya, yb):
        "boundary condition residual"
        return np.array([ya[0] - a, yb[0] - b, yb[2]-3])
    
    
    # solve bvp
    sol = solve_bvp(f, bc, x, y0)
    plot.update_traces(x=sol.x, y=sol.y[0, :], selector=dict(name='y0'))
    plot.update_traces(x=sol.x, y=sol.y[1, :], selector=dict(name='y1'))
    plot.update_traces(x=sol.x, y=sol.y[2, :], selector=dict(name='y2'))
    plot.show()

wdg.interact(test_bvp,
             plot=wdg.fixed(test_plot),
             a=5.,
             b=4.,
             r=1.);

interactive(children=(FloatSlider(value=5.0, description='a', max=15.0, min=-5.0), FloatSlider(value=4.0, desc…

In [25]:
test_plot = go.Figure()
test_plot.add_trace(go.Scatter(name='SO4'))
test_plot.add_trace(go.Scatter(name='dSO4'))
test_plot.add_trace(go.Scatter(name='py'))
test_plot.update_layout(width=700, height=500)

def test_bvp(plot, D, v, k_MSR, SO4_0, py_0):
    
#     def f(x, y):
#         "right hand side of ODE"
#         return np.vstack([y[1,:],
#                           r*y[0,:],
#                           y[1,:]*y[0,:]])
    
    n = 100 # number of points
    x = np.linspace(0,3.2,n)
    y0 = np.zeros((3,n))
    
    def f(x, y):
        '''The system of ODEs that need to be solved to get the steady state concentrations'''
        return np.vstack([
               y[1,:], # = y[0]' = d[SO₄]/dz (Eq. a)
               (v*y[1,:] + k_MSR*y[0,:])/D, # = y[1]' = d²[SO₄]/dz² (Eq. b)
               k_MSR * y[0,:] / (2*v) # = y[2]' = d[py]/dz
              ])

    def bc(ya, yb):
        "boundary condition residual"
        return np.array([ya[0] - SO4_0, yb[1] - 0, ya[2] - py_0])
    
    
    # solve bvp
    sol = solve_bvp(f, bc, x, y0)
    plot.update_traces(x=sol.x, y=sol.y[0, :], selector=dict(name='SO4'))
    plot.update_traces(x=sol.x, y=sol.y[1, :], selector=dict(name='dSO4'))
    plot.update_traces(x=sol.x, y=sol.y[2, :], selector=dict(name='py'))
    plot.show()

wdg.interact(test_bvp,
             plot=wdg.fixed(test_plot),
             D=1.,
             v=1.,
             k_MSR=1.,
             SO4_0=1.,
             py_0=1.);

interactive(children=(FloatSlider(value=1.0, description='D', max=3.0, min=-1.0), FloatSlider(value=1.0, descr…